In [263]:
import logging
import yaml

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

import SALPY_ATPtg
import SALPY_ATMCS

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False



In [3]:
import os
print(os.environ["LSST_DDS_DOMAIN"])

lsatmcs


In [4]:
d = salobj.Domain()

In [5]:
r1 = salobj.Remote(d, "ATPtg")

In [6]:
await r1.start_task

In [12]:
r2 = salobj.Remote(d, "ATMCS")

In [13]:
await r2.start_task

In [ ]:
await salobj.set_summary_state(r2, salobj.State.ENABLED)

In [ ]:
r3 = salobj.Remote(d, "ATDomeTrajectory")

In [ ]:
await r3.start_task

In [ ]:
await salobj.set_summary_state(r3, salobj.State.ENABLED)

In [ ]:
r4 = salobj.Remote(d, "ATPneumatics")

In [ ]:
r5 = salobj.Remote(d, "ATDome", index=1)

In [ ]:
await r5.start_task

In [ ]:
await r5.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(r5, salobj.State.ENABLED, settingsToApply='test')

In [ ]:
await salobj.set_summary_state(r5, salobj.State.STANDBY)

In [ ]:
await r5.cmd_homeAzimuth.start()

In [ ]:
await r5.cmd_openShutter.start()

In [ ]:
error = await r5.evt_errorCode.next(flush=False, timeout=5)

In [ ]:
print(error.errorReport)

In [ ]:
await asyncio.sleep(5)
print(salobj.State(r2.evt_summaryState.get().summaryState))

In [ ]:
await r2.cmd_disable.start(timeout=30)

In [ ]:
await r5.cmd_openShutter.start()

In [ ]:
await r1.start_task
await r2.start_task

In [ ]:
r1 = salobj.Remote(SALPY_ATPtg)
r2 = salobj.Remote(SALPY_ATMCS)

In [ ]:
await r1.tel_timeAndDate.next(flush=True)

In [ ]:
# r1.evt_summaryState.flush()
await salobj.set_summary_state(r1, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(r1, salobj.State.ENABLED)

In [ ]:
await r2.cmd_start.start(timeout=30)

In [ ]:
await r2.cmd_enable.start(timeout=30)

In [ ]:
await r2.cmd_disable.start(timeout=30)

In [ ]:
await asyncio.sleep(5)

In [ ]:
await salobj.set_summary_state(r2, salobj.State.DISABLED, timeout=60)

In [ ]:
print('reading...')
await asyncio.sleep(10.)
print('working...')
res = await salobj.set_summary_state(r2, salobj.State.ENABLED, timeout=60)
print(res)
print('done')

In [ ]:
print('reading...')
await asyncio.sleep(10.)
print('working...')
res = await salobj.set_summary_state(r2, salobj.State.ENABLED, timeout=60)
print(res)
print('done')

In [ ]:
print(salobj.State(r1.evt_summaryState.get().summaryState))
print(salobj.State(r2.evt_summaryState.get().summaryState))

In [ ]:
await salobj.set_summary_state(r2, salobj.State.ENABLED, timeout=300)

In [ ]:
await salobj.set_summary_state(r1, salobj.State.ENABLED, 
                               timeout=300)
# await salobj.set_summary_state(r2, salobj.State.ENABLED, timeout=300)

In [ ]:
print(salobj.State(r1.evt_summaryState.get().summaryState))
print(salobj.State(r2.evt_summaryState.get().summaryState))

In [162]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                                    lat=-30.244728*u.deg,
                                                    height=2663.0*u.m)

In [ ]:
time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=70.*u.deg, az=0.*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Compute rot_pa
lst = Angle(time_data.lst, unit=u.hourangle)
lha = lst - cmd_radec.ra
# rot_pa = Angle(palpy.pa(lha.radian, cmd_radec.dec.radian, location.lat.radian)*u.radian)
# print(f"rotPA: {rot_pa.deg}")

# Start tracking
r1.cmd_raDecTarget.set(
    targetName="atptg_atmcs_integration",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-cmd_elaz.alt.deg,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")
# script.ataos.cmd_enableCorrection.set(hexapod=True)
# await script.ataos.cmd_enableCorrection.start(timeout=10)



In [ ]:
Angle('16:29:25', unit=u.hour), Angle('-26:25:55', unit=u.deg)
Angle('16:54:02', unit=u.hour), Angle('-22:06:20', unit=u.deg)

In [ ]:
Angle('16:29:25', unit=u.hour).hour

In [ ]:
ICRS

In [ ]:
r1.cmd_raDecTarget.set(
    targetName="HR 7235",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2019.56,  # should be ignored for ICRS
    ra=Angle('19:05:24', unit=u.hour).hour,
    declination=Angle('+13:51:47', unit=u.deg).deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

In [ ]:
r1.cmd_raDecTarget.set(
    targetName="HR 6527",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2019.5,  # should be ignored for ICRS
    ra=Angle('17:33:36', unit=u.hour).hour,
    declination=Angle('-37:06:12', unit=u.deg).deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-57.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

In [ ]:
dra = 0.24*24./360.
ddec= -0.09
print(dra)

In [ ]:
# edge 1
dra = 0.24*24./360.
ddec= 0.05
print(dra)

In [ ]:
# edge 1 ------ this one
dra = 0.29*24./360.
ddec= 0.08
print(dra)

In [ ]:
# edge 2 ------ this one
dra = 0.48*24./360.
ddec= -0.36
print(dra)

In [69]:
import numpy as np
dra = (0.24+0.48)/2.*24./360.
ddec = (0.08-0.36)/2.
print(np.sqrt( ((0.24+0.48)/2.)**2 + ddec**2))
print((0.24+0.48)/2.)
print(ddec)

0.3862641583165593
0.36
-0.13999999999999999


In [ ]:
r1.cmd_raDecTarget.set(
    targetName="Moon",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=Angle('02:18:08', unit=u.hour).hour+dra,
    declination=Angle('8:29:15', unit=u.deg).deg+ddec,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-30.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

In [ ]:
dra = ((0.24+0.48)+0.0)/2.*24./360.
ddec = ((0.08-0.36)+0.0)/2.

In [ ]:
dra = ((0.24+0.48)+0.035)/2.*24./360.
ddec = ((0.08-0.36)+0.03)/2.

In [ ]:
dra = ((0.24+0.48)+0.055)/2.*24./360.
ddec = ((0.08-0.36)+0.05)/2.

In [ ]:
dra = ((0.24+0.48)+0.065)/2.*24./360.
ddec = ((0.08-0.36)+0.12)/2.

In [ ]:
r1.cmd_raDecTarget.set(
    targetName="HIP 113368",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=Angle('22:57:39', unit=u.hour).hour+dra,
    declination=Angle('-29:37:20', unit=u.deg).deg+ddec,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-30.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

In [ ]:
dra = ((0.24+0.48)+0.055)/2.*24./360.
ddec = ((0.08-0.36)+0.22)/2.

In [ ]:
r1.cmd_raDecTarget.set(
    targetName="Achernar",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=Angle('01:37:42', unit=u.hour).hour+dra,
    declination=Angle('-57:14:11', unit=u.deg).deg+ddec,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-30.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

In [ ]:
dra = ((0.24+0.48)+0.055)/2.*24./360.
ddec = ((0.08-0.36)+0.22)/2.

In [ ]:
r1.cmd_raDecTarget.set(
    targetName="Canopus",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=Angle('6:23:57', unit=u.hour).hour+dra,
    declination=Angle('-52:41:44', unit=u.deg).deg+ddec,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-30.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

In [ ]:
# r2.evt_target.flush()
# r2.evt_allAxesInPosition.flush()

ack_id = await r1.cmd_raDecTarget.start(timeout=60)
print(f"raDecTarget command result: {ack_id.ack}")

# while True:
#     in_position = await r2.evt_allAxesInPosition.next(flush=False)
#     print(f"Got {in_position.inPosition}")
#     if in_position.inPosition:
#         break

In [ ]:
ack_id = await r1.cmd_stopTracking.start(timeout=30)

In [ ]:
ack = await r1.cmd_offsetAzEl.set_start(az=-0.025*60.*60.) ### THIS ONE
print(ack.ack,ack.error, ack.result)

In [ ]:
ack = await r1.cmd_offsetAzEl.set_start(az=-0.3*60.*60.)
print(ack.ack, ack.error, ack.result)

In [ ]:
ack = await r1.cmd_offsetAzEl.set_start(el=+0.24*60.*60.) 
print(ack.ack, ack.error, ack.result)

In [ ]:
ack = await r1.cmd_offsetAzEl.set_start(el=0.02*60.*60., az=0.02*60.*60.)### THIS ONE
print(ack.ack, ack.error, ack.result)

In [ ]:
ack = await r1.cmd_offsetAzEl.set_start(el=0.1*60.*60., az=-0.27*60.*60.)### THIS ONE
print(ack.ack, ack.error, ack.result)

In [ ]:
# while True:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)
print(demand.demandAz, demand.demandEl, demand.demandRot)
target = await r2.evt_target.next(flush=True, timeout=20)
print(target.azimuth, target.elevation, target.nasmyth1RotatorAngle)

In [ ]:
from math import isclose

In [ ]:
isclose?

In [ ]:
time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=40.*u.deg, az=285.*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Start tracking
r1.cmd_raDecTarget.set(
    targetName="atptg_atmcs_integration",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=-180.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_target,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")
# script.ataos.cmd_enableCorrection.set(hexapod=True)
# await script.ataos.cmd_enableCorrection.start(timeout=10)


In [ ]:
ack_id = await r2.cmd_startTracking.start(timeout=5, wait_done=False)
print(f"startTracking: {ack_id.ack.ack}::{ack_id.ack.result}")

In [ ]:
await r1.cmd_planetTarget.set_start(planetName='Jupiter')

In [ ]:
target = await r2.evt_target.next(flush=True, timeout=5)
print(target.azimuth, target.elevation, target.nasmyth1RotatorAngle)

In [ ]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)
demand.demandAz, demand.demandEl, demand.demandRot

In [ ]:
# demand = await r1.evt_trackPosting

In [ ]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)
demand.demandAz, demand.demandEl, demand.demandRot

In [ ]:
ack_id = await r2.cmd_stopTracking.start(timeout=30)

In [ ]:
print(salobj.State(r1.evt_summaryState.get().summaryState))
print(salobj.State(r2.evt_summaryState.get().summaryState))

In [ ]:
def atptg_target_callback(id_data):
    print(id_data.demandAz, id_data.demandEl)

# def atmcs_demand_callback(id_data):
#     print(id_data.)

In [ ]:
r1.tel_currentTargetStatus.callback = atptg_target_callback

In [ ]:
await asyncio.sleep(10)

In [ ]:
r1.tel_currentTargetStatus.callback = None

In [ ]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)

In [ ]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)
demand.demandAz, demand.demandEl, demand.demandRot

In [ ]:
for i in range(10):
    demand = await r2.evt_target.next(flush=True, timeout=5)
    print(demand.elevation, demand.azimuth)

In [ ]:
ack_id = await r2.cmd_stopTracking.start(timeout=10)

In [ ]:
await salobj.set_summary_state(r1, salobj.State.STANDBY, 
                               timeout=300)
# await salobj.set_summary_state(r2, salobj.State.STANDBY, timeout=300)

In [ ]:
import os

In [ ]:
os.environ["LSST_DDS_DOMAIN"]

In [ ]:
await r1.cmd_exitControl.start()

In [ ]:
50e-3*200

In [ ]:
import palpy

In [ ]:
palpy.pa?

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                                    lat=-30.244728*u.deg,
                                                    height=2663.0*u.m)

In [ ]:
time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=45.*u.deg, az=0.*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)



In [ ]:
cmd_radec.ra.hour,time_data.lst

In [ ]:
lst = Angle(time_data.lst, unit=u.hour)

In [ ]:
lha = lst - cmd_radec.ra

In [ ]:
lha.hour

In [ ]:
rot_pa = Angle(palpy.pa(lha.radian, cmd_radec.dec.radian, location.lat.radian)*u.radian)

In [ ]:
rot_pa.deg

In [ ]:
rot_pa.deg

In [ ]:
palpy.pa?

In [ ]:
1./200e-3/25

In [ ]:
1./25e-3

In [ ]:
await r3.start_task

In [ ]:
await salobj.set_summary_state(r3, salobj.State.DISABLED)

In [ ]:
await r2.cmd_setInstrumentPort.set_start(port=1)

In [ ]:
await  r4.start_task

In [ ]:
await r4.cmd_closeM1Cover.start()

In [ ]:
await r5.start_task

In [ ]:
await r5.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await r5.cmd_closeShutter.start(timeout=10)

In [ ]:
await asyncio.sleep(10)
print(salobj.State(r5.evt_summaryState.get().summaryState))



In [ ]:
await r5.cmd_disable.start(timeout=5)

In [ ]:
rrra=Angle(263.4*u.deg)
dddec=Angle(-37.1*u.deg)

# 20190725

In [7]:
dra = ((0.24+0.48)+0.055)/2.*24./360.
ddec = ((0.08-0.36)+0.22)/2.

In [10]:
r1.cmd_raDecTarget.set(
    targetName="Alpha Centauri",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=Angle('14:39:36', unit=u.hour).hour+dra,
    declination=Angle('-60 50 02', unit=u.deg).deg+ddec,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-59.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

raDecTarget ra=14.685833333333333 hour; declination=-60.863888888888894 deg


In [156]:
dra = -0.35*24./360.
ddec = -.55

In [159]:
dra = -0.35*24./360.
ddec = -.53

In [215]:
dra = -0.29*24./360.
ddec = -.57

In [216]:

r1.cmd_raDecTarget.set(
    targetName="Jupiter",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=Angle('16:53:37', unit=u.hour).hour+dra,
    declination=Angle('-22:06:01', unit=u.deg).deg+ddec,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

raDecTarget ra=16.874277777777777 hour; declination=-22.67027777777778 deg


In [212]:
dra = -0.5*24./360.
ddec = -.7

In [213]:

r1.cmd_raDecTarget.set(
    targetName="Eta Oph",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=Angle('17:10:22.7 ', unit=u.hour).hour+dra,
    declination=Angle('-15:43:30.', unit=u.deg).deg+ddec,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

raDecTarget ra=17.13963888888889 hour; declination=-16.425 deg


In [217]:
await r1.cmd_raDecTarget.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [187]:
await r1.cmd_stopTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 11 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


raDecTarget ra=16.470794361111114 hour; declination=-27.002002611111113 deg


In [17]:
await r2.cmd_startTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling beh

# GRID

In [90]:
dra_grid = np.arange(-1., 1.,0.1)*24./360.
ddec_grid = np.arange(-1., 1.,0.1)

In [91]:
dra_grid

array([-6.66666667e-02, -6.00000000e-02, -5.33333333e-02, -4.66666667e-02,
       -4.00000000e-02, -3.33333333e-02, -2.66666667e-02, -2.00000000e-02,
       -1.33333333e-02, -6.66666667e-03, -1.48029737e-17,  6.66666667e-03,
        1.33333333e-02,  2.00000000e-02,  2.66666667e-02,  3.33333333e-02,
        4.00000000e-02,  4.66666667e-02,  5.33333333e-02,  6.00000000e-02])

In [80]:
async def wait_center():
    while True:
        opt = input("Center telescope and choose action (press ?<enter>, for list of actions): ")
        
        if opt == 'x':
            print("Telescope centered. Adding data and continuing...")
            await atptg.cmd_pointAddData.start()
            return 0
        elif opt == 's':
            print("Skipping...")
            return 1
        elif opt == 't':
            print("Stopping test...")
            return -1
        elif opt == '?':
            print("""Options are:
            x - Telescope centered, add data and continue.
            s - Skipp point and continue.
            t - Terminate test.
            """)

In [92]:
for i in range(len(dra_grid)):
    for j in range(len(ddec_grid)):
        r1.cmd_raDecTarget.set(
            targetName="Jupiter",
            targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
            frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
            epoch=2000,  # should be ignored: no parallax or proper motion
            equinox=2000,  # should be ignored for ICRS
            ra=Angle('16:53:37', unit=u.hour).hour+dra_grid[i],
            declination=Angle('-22:06:01', unit=u.deg).deg+ddec_grid[i],
            parallax=0,
            pmRA=0,
            pmDec=0,
            rv=0,
            dRA=0,
            dDec=0,
            rotPA=180.-74.,
            rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
            rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field)
        print(f"GRID[{i}][{j}]")
        print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour;"
              f"declination={r1.cmd_raDecTarget.data.declination!r} deg")
        await r1.cmd_raDecTarget.start()
        await wait_center()


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


GRID[0][0]
raDecTarget ra=16.826944444444443 hour;declination=-23.10027777777778 deg


Center telescope and choose action (press ?<enter>, for list of actions):  t


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Stopping test...
GRID[0][1]
raDecTarget ra=16.826944444444443 hour;declination=-23.10027777777778 deg


KeyboardInterrupt: 

In [82]:
await r1.cmd_stopTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [194]:
0.05*60*60/2+360


450.0

In [242]:
await r2.cmd_startTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling beh

COORD.

In [219]:
radec = ICRS((Angle('16:53:37', unit=u.hour).hour+dra)*u.hour,
             (Angle('-22:06:01', unit=u.deg).deg+ddec)*u.deg)

In [220]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                                    lat=-30.244728*u.deg,
                                                    height=2663.0*u.m)

In [221]:
time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [224]:
alt_az = radec.transform_to(aa)

In [226]:
alt_az.alt += 

AttributeError: Cannot set any frame attribute alt

In [227]:
cmd_elaz = AltAz(alt=alt_az.alt+0.0519*u.deg, az=alt_az.az+2.133*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

In [231]:
cmd_radec.ra.hour, cmd_radec.dec.deg

(16.89599026205912, -22.71479215452344)

In [ ]:
alt_az = radec.transform_to(aa)

In [252]:
dalt = 0. #-0.0519
daz = 0. #-2.133

radec = ICRS(Angle('16:53:37', unit=u.hour),
             Angle('-22:06:01', unit=u.deg))

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt*u.deg, az=alt_az.az+daz*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

r1.cmd_raDecTarget.set(
    targetName="Jupiter",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

mount_data = await r2.tel_mountEncoders.next(flush=True, timeout=5)

print(f"Alt: {alt_az.alt}")
print(f"Az: {alt_az.az-360.*u.deg}")
print(f"delta_Alt: {alt_az.alt - mount_data.elevationCalculatedAngle*u.deg}")
print(f"delta_Az: {alt_az.az-360.*u.deg - mount_data.azimuthCalculatedAngle*u.deg}")
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

dalt = alt_az.alt - mount_data.elevationCalculatedAngle*u.deg
daz = alt_az.az-360.*u.deg - mount_data.azimuthCalculatedAngle*u.deg
# await cmd_raDecTarget.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Alt: 81.74882239463997 deg
Az: -10.841210788961689 deg
delta_Alt: 3.176938571397585 deg
delta_Az: 39.38358216661254 deg
raDecTarget ra=17.45722500312946 hour; declination=-22.1159650602712 deg


In [256]:
dalt = -3.176938571397585*u.deg
daz = -39.38358216661254*u.deg

In [257]:
radec = ICRS(Angle('17:10:22.7', unit=u.hour),
             Angle('-15:43:29.7', unit=u.deg))

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt, az=alt_az.az+daz, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Eta Ophi",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 11 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [392]:
dalt = 3.176938571397585*u.deg
daz = 39.38358216661254*u.deg

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)
print(mount.azimuthCalculatedAngle, mount.elevationCalculatedAngle)
print(289. + 50./60 -360 - mount.azimuthCalculatedAngle )
print(69 + 47/60. - mount.elevationCalculatedAngle)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


-72.96754449431319 68.64183955965564
2.8008778276465023
1.141493773677695


In [400]:
await r2.cmd_startTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 19 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 

In [ ]:
dalt = (68 + 38/60. - mount.elevationCalculatedAngle)*u.deg
daz = -(286. + 27./60 -360 - mount.azimuthCalculatedAngle)*u.deg

In [399]:
dalt, daz

(<Quantity 0.28839867 deg>, <Quantity 0.17255585 deg>)

In [423]:
dalt2, daz2

(<Quantity 0.65039867 deg>, <Quantity 0.25255585 deg>)

In [422]:
radec = FK5(Angle('16:53:37', unit=u.hour),
             Angle('-22:06:01', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt2 = dalt+0.362*u.deg
daz2 = daz + 0.08*u.deg

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt2, az=alt_az.az+daz2, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(FK5)


r1.cmd_raDecTarget.set(
    targetName="Jupiter",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling beh

dra: -0.04521008052153164
dec: 0.1954086961896948


In [443]:
await r1.cmd_stopTracking.start()

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 92 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 46 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [441]:
radec = ICRS(Angle('16:29:25', unit=u.hour),
             Angle('-26:25:55', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt3 = dalt2+0.0*u.deg
daz3 = daz2+0.08*u.deg

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt3, az=alt_az.az+daz3, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Sco",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling beh

dra: -0.05009861802799165
dec: 0.006283431668073547


# After offset

In [446]:
261-360

-99

In [445]:
radec = ICRS(Angle('16:29:25', unit=u.hour),
             Angle('-26:25:55', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt4 = 0.
daz4 = 0.

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt4, az=alt_az.az+daz4, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Sco",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


dra: -4.817479748453479e-12
dec: 3.070255161219393e-11


In [455]:
dalt4,daz4

(<Quantity 0.15 deg>, <Quantity 0.3 deg>)

In [459]:
radec = FK5(Angle('16:53:37', unit=u.hour),
             Angle('-22:06:01', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt5 = dalt4+0.0*u.deg
daz5 = daz4+0.1*u.deg

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt5, az=alt_az.az+daz5, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(FK5)


r1.cmd_raDecTarget.set(
    targetName="Jupiter",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-47.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling beh

dra: -0.01758754321728162
dec: -0.21186376605500357


In [442]:
daz3, dalt3

(<Quantity 0.33255585 deg>, <Quantity 0.65039867 deg>)

In [264]:
dalt = 0. #-0.0519
daz = 0. #-2.133

radec = ICRS(Angle('17:10:22.7', unit=u.hour),
             Angle('-15:43:29.7', unit=u.deg))

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt*u.deg, az=alt_az.az+daz*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

r1.cmd_raDecTarget.set(
    targetName="Jupiter",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

mount_data = await r2.tel_mountEncoders.next(flush=True, timeout=5)

print(f"Alt: {alt_az.alt}")
print(f"Az: {alt_az.az-360.*u.deg}")
print(f"delta_Alt: {alt_az.alt - mount_data.elevationCalculatedAngle*u.deg}")
print(f"delta_Az: {alt_az.az - mount_data.azimuthCalculatedAngle*u.deg}")
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

dalt = alt_az.alt - mount_data.elevationCalculatedAngle*u.deg
daz = alt_az.az - mount_data.azimuthCalculatedAngle*u.deg
# await cmd_raDecTarget.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Alt: 75.32088472157214 deg
Az: -350.43123983852627 deg
delta_Alt: -9.946126737864475 deg
delta_Az: -9.550928707791057 deg
raDecTarget ra=17.172972222222224 hour; declination=-15.724916666666667 deg


In [ ]:
r1.cmd_raDecTarget.set(
    targetName="Eta Ophi",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

await r1.cmd_raDecTarget.start(timeout=30.)

In [235]:
time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")

falling behind; queue contains 13 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [ ]:
cmd_radec = cmd_elaz.transform_to(ICRS)

In [ ]:

r1.cmd_raDecTarget.set(
    targetName="Eta Ophi",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=Angle('16:53:37', unit=u.hour).hour+dra,
    declination=Angle('-22:06:01', unit=u.deg).deg+ddec,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-74.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")

# GRID

In [481]:
alt_grid = np.linspace(-0.5, .5,7)
az_grid = np.linspace(-.5, .5,7)

In [451]:
print(alt_grid)

[-0.3  -0.15  0.    0.15  0.3 ]


In [482]:
for i in range(1,len(alt_grid)):
    for j in range(len(az_grid)):
        
        print(f"{i} {j}")
        
        radec = FK5(Angle('16:53:37', unit=u.hour),
                     Angle('-22:06:01', unit=u.deg))

        mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

        dalt4 = 0.333*2*u.deg+alt_grid[i]*u.deg
        daz4 = 0.650*2*u.deg+az_grid[j]*u.deg

        

        time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
        curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
        aa = AltAz(location=location, obstime=curr_time_atptg)
        alt_az = radec.transform_to(aa)

        cmd_elaz = AltAz(alt=alt_az.alt+dalt4, az=alt_az.az+daz4, 
                         obstime=curr_time_atptg.tai, 
                         location=location)
        cmd_radec = cmd_elaz.transform_to(FK5)


        r1.cmd_raDecTarget.set(
            targetName="Jupiter",
            targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
            frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
            epoch=2000,  # should be ignored: no parallax or proper motion
            equinox=2000,  # should be ignored for ICRS
            ra=cmd_radec.ra.hour,
            declination=cmd_radec.dec.deg,
            parallax=0,
            pmRA=0,
            pmDec=0,
            rv=0,
            dRA=0,
            dDec=0,
            rotPA=180.-47.,
            rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
            rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
        )

        print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


        print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

        
        await r1.cmd_raDecTarget.start()
        await wait_center()



falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


1 0
dra: -0.038527829312911877
dec: -0.44270827896091447


Center telescope and choose action (press ?<enter>, for list of actions):  s


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Skipping...
1 1
dra: -0.0420082255616272
dec: -0.5618300200279833


...

Skipping...
5 6
dra: -0.10354110371489611
dec: -0.9007204136919178


Center telescope and choose action (press ?<enter>, for list of actions):  s


AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 12 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 13 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 13 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 52 messages
falling behind; queue contains 56 elements
falling behind;

Skipping...
6 0
dra: -0.09418094806886401
dec: -0.13521594837736117


KeyboardInterrupt: 

In [495]:
dalt4,daz4

(<Quantity 1.316 deg>, <Quantity 0.73 deg>)

In [494]:
radec = ICRS(Angle('16:53:35', unit=u.hour),
             Angle('-22:05:16', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt4 = 0.333*2*u.deg+alt_grid[6]*u.deg+0.15*u.deg
daz4 = 0.650*2*u.deg+az_grid[0]*u.deg-0.07*u.deg

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt4, az=alt_az.az+daz4, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Jupiter",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-47.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")


await r1.cmd_raDecTarget.start()

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue co

dra: -0.10311089380458682
dec: -0.037369333709307284


In [501]:
alt_grid

array([-0.5       , -0.33333333, -0.16666667,  0.        ,  0.16666667,
        0.33333333,  0.5       ])

In [510]:
print(dalt4, daz4)
print(dalt6, daz6)

1.3159999999999998 deg 0.73 deg
1.3159999999999998 deg 0.745 deg


In [508]:
radec = ICRS(Angle('16:29:25', unit=u.hour),
             Angle('-26:25:55', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt6 = dalt4+alt_grid[3]*u.deg
daz6 = daz4+az_grid[3]*u.deg+0.015*u.deg


time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt6, az=alt_az.az+daz6, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Sco",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 50 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 50 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 50 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 50 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 50 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 50 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 50 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 50 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 50 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind

dra: -0.10804928824185467
dec: -0.12459301924713628


In [500]:
for i in range(1,len(alt_grid)):
    for j in range(len(az_grid)):
        
        print(f"{i} {j}")
        
        radec = ICRS(Angle('16:29:25', unit=u.hour),
                     Angle('-26:25:55', unit=u.deg))


        mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

        dalt5 = dalt4+alt_grid[i]*u.deg
        daz5 = daz4+az_grid[j]*u.deg

        

        time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
        curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
        aa = AltAz(location=location, obstime=curr_time_atptg)
        alt_az = radec.transform_to(aa)

        cmd_elaz = AltAz(alt=alt_az.alt+dalt5, az=alt_az.az+daz5, 
                         obstime=curr_time_atptg.tai, 
                         location=location)
        cmd_radec = cmd_elaz.transform_to(FK5)


        r1.cmd_raDecTarget.set(
            targetName="Alpha Sco",
            targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
            frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
            epoch=2000,  # should be ignored: no parallax or proper motion
            equinox=2000,  # should be ignored for ICRS
            ra=cmd_radec.ra.hour,
            declination=cmd_radec.dec.deg,
            parallax=0,
            pmRA=0,
            pmDec=0,
            rv=0,
            dRA=0,
            dDec=0,
            rotPA=180.-47.,
            rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
            rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
        )

        print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


        print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

        
        await r1.cmd_raDecTarget.start()
        await wait_center()




falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 14 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


1 0
dra: -0.07362309631615105
dec: 0.15950533697757052


Center telescope and choose action (press ?<enter>, for list of actions):  s


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Skipping...
1 1
dra: -0.07716072913696337
dec: 0.030603429687385386


...

Skipping...
3 2
dra: -0.10403655238818033
dec: 0.016242783834215402


Center telescope and choose action (press ?<enter>, for list of actions):  s


AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 16 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 16 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 16 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 16 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 16 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 16 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 65 messages
falling behind; queue contains 94 elements
falling behind; queue contains 93 elements
falling behind; queue contains 92 elements


Skipping...
3 3
dra: -0.1075828270945145
dec: -0.11334746129725204


KeyboardInterrupt: 

In [466]:
i=0 
j=6
print(f"{i} {j}")

radec = FK5(Angle('16:53:37', unit=u.hour),
             Angle('-22:06:01', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt4 = alt_grid[i]*u.deg
daz4 = az_grid[j]*u.deg

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt4, az=alt_az.az+daz4, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(FK5)


r1.cmd_raDecTarget.set(
    targetName="Jupiter",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_fk5,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-47.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")


await r1.cmd_raDecTarget.start()

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 13 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


0 6
dra: 0.0909942303750384
dec: -2.1535843381292956


In [468]:
await r2.cmd_startTracking.start()

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 14 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, target) f

# Alpha Grus

In [512]:
radec = ICRS(Angle('22:08:13.98473', unit=u.hour),
             Angle('-46:57:39.5078', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt7 = dalt6+alt_grid[3]*u.deg
daz7 = daz6+az_grid[3]*u.deg+0.015*u.deg


time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt7, az=alt_az.az+daz7, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Grus",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


dra: 0.1047995941854154
dec: -0.7957410173831221


In [514]:
for i in range(1,len(alt_grid)):
    for j in range(len(az_grid)):
        
        print(f"{i} {j}")
        
        radec = ICRS(Angle('22:08:13.98473', unit=u.hour),
                     Angle('-46:57:39.5078', unit=u.deg))



        mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

#         dalt5 = dalt4+alt_grid[i]*u.deg
#         daz5 = daz4+az_grid[j]*u.deg
        dalt7 = dalt6+alt_grid[i]*u.deg
        daz7 = daz6+az_grid[j]*u.deg


        
        time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
        curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
        aa = AltAz(location=location, obstime=curr_time_atptg)
        alt_az = radec.transform_to(aa)

        cmd_elaz = AltAz(alt=alt_az.alt+dalt7, az=alt_az.az+daz7, 
                         obstime=curr_time_atptg.tai, 
                         location=location)
        cmd_radec = cmd_elaz.transform_to(FK5)


        r1.cmd_raDecTarget.set(
            targetName="Alpha Grus",
            targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
            frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
            epoch=2000,  # should be ignored: no parallax or proper motion
            equinox=2000,  # should be ignored for ICRS
            ra=cmd_radec.ra.hour,
            declination=cmd_radec.dec.deg,
            parallax=0,
            pmRA=0,
            pmDec=0,
            rv=0,
            dRA=0,
            dDec=0,
            rotPA=180.-44.,
            rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
            rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
        )

        print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


        print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

        
        await r1.cmd_raDecTarget.start()
        await wait_center()





falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


1 0
dra: 0.0686902410825958
dec: -0.6869502231793447


Center telescope and choose action (press ?<enter>, for list of actions):  s


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Skipping...
1 1
dra: 0.07193142368180361
dec: -0.6605773157534074


...

Skipping...
3 1
dra: 0.09130796180064493
dec: -0.9285190828166137


Center telescope and choose action (press ?<enter>, for list of actions):  s


AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 13 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 13 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 53 messages
falling behind; queue contains 60 elements
falling behind; queue contains 59 elements


Skipping...
3 2
dra: 0.09532615190907023
dec: -0.8951692171685863


KeyboardInterrupt: 

In [536]:
dalt8.to(u.arcsec), daz8

(<Quantity 4557.6 arcsec>, <Quantity 0.47833333 deg>)

In [550]:
radec = ICRS(Angle('22:08:13.98473', unit=u.hour),
             Angle('-46:57:39.5078', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt8 = dalt6+alt_grid[3]*u.deg-0.05*u.deg
daz8 = daz6+az_grid[2]*u.deg-0.10*u.deg

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt8, az=alt_az.az+daz8, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Grus",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 83 messages
falling behind; queue contains 82 elements
falling behind; queue contains 81 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 16 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 27 messages


dra: 0.028648447812692268
dec: -1.2379074128102658


In [665]:
radec = ICRS(Angle('22:08:13.98473', unit=u.hour),
             Angle('-46:57:39.5078', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt8 = dalt6+alt_grid[3]*u.deg-0.05*u.deg
daz8 = daz6+az_grid[2]*u.deg-0.20*u.deg

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt8, az=alt_az.az+daz8, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Grus",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

ddra = radec.ra-cmd_radec.ra
dddec = radec.dec-cmd_radec.dec
print(ddra.to(u.arcsec))
print(dddec.to(u.arcsec))
print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


-1715.39arcsec
-4418.73arcsec
dra: -0.0317664399956179
dec: -1.2274252698813086


In [661]:
dalt8.to(u.arcsec).value, daz8.to(u.arcsec).value

(4557.599999999999, 1362.0)

In [664]:
await r1.cmd_offsetAzEl.set_start(az=362., el=0., num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [675]:
await r1.cmd_offsetRADec.set_start(off1=100, off2=0., num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [617]:
await r1.cmd_offsetAzEl.set_start(az=0., el=+557.6, num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling beh

In [618]:
await r2.cmd_startTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 14 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 55 elements
falling behind; queue contains 54 elements


# Alpha Eri

In [694]:
for i in range(1,len(alt_grid)):
    for j in range(len(az_grid)):
        
        print(f"{i} {j}")
        
        radec = ICRS(Angle('01:37:42.84548', unit=u.hour),
                     Angle('-57:14:12.3101', unit=u.deg))

        mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

#         dalt5 = dalt4+alt_grid[i]*u.deg
#         daz5 = daz4+az_grid[j]*u.deg
        dalt9 = dalt8+alt_grid[i]*u.deg
        daz9 = daz8+az_grid[j]*u.deg


        
        time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
        curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
        aa = AltAz(location=location, obstime=curr_time_atptg)
        alt_az = radec.transform_to(aa)

        cmd_elaz = AltAz(alt=alt_az.alt+dalt9, az=alt_az.az+daz9, 
                         obstime=curr_time_atptg.tai, 
                         location=location)
        cmd_radec = cmd_elaz.transform_to(FK5)


        r1.cmd_raDecTarget.set(
            targetName="Alpha Eri",
            targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
            frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
            epoch=2000,  # should be ignored: no parallax or proper motion
            equinox=2000,  # should be ignored for ICRS
            ra=cmd_radec.ra.hour,
            declination=cmd_radec.dec.deg,
            parallax=0,
            pmRA=0,
            pmDec=0,
            rv=0,
            dRA=0,
            dDec=0,
            rotPA=180.-44.,
            rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
            rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
        )

        print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


        print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

        
        await r1.cmd_raDecTarget.start()
        await wait_center()






falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 23 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


1 0
dra: 0.09896188821151042
dec: -0.4714015094080892


Center telescope and choose action (press ?<enter>, for list of actions):  s


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Skipping...
1 1
dra: 0.10422040637008156
dec: -0.38414830433944047


...

Skipping...
3 4
dra: 0.15702577243016091
dec: -0.2820090362757597


Center telescope and choose action (press ?<enter>, for list of actions):  s


RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 19 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 19 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 19 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 19 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 78 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 35 elements


Skipping...
3 5
dra: 0.1629783943481693
dec: -0.193299797839245


KeyboardInterrupt: 

In [705]:
radec = ICRS(Angle('01:37:42.84548', unit=u.hour),
             Angle('-57:14:12.3101', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt10 = dalt8+alt_grid[3]*u.deg-0.025*u.deg
daz10 = daz8+az_grid[5]*u.deg-0.025*u.deg

time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt10, az=alt_az.az+daz10, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Eri",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

ddra = radec.ra-cmd_radec.ra
dddec = radec.dec-cmd_radec.dec
print(ddra.to(u.arcsec))
print(dddec.to(u.arcsec))
print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 54 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 54 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue cont

8535.59arcsec
-865.772arcsec
dra: 0.15806645402975694
dec: -0.24049220666104532


In [719]:
await r1.cmd_offsetAzEl.set_start(az=0., el=30., num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [706]:
await r1.cmd_offsetRADec.set_start(off1=-1000, off2=814.614, num=1)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 90 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 90 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 90 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 90 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 90 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 90 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 90 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [714]:
await r1.cmd_offsetRADec.set_start(off1=-1000, off2=0., num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


# Alpha Pav

In [721]:
for i in range(1,len(alt_grid)):
    for j in range(len(az_grid)):
        
        print(f"{i} {j}")
        
        radec = ICRS(Angle('20:25:38.85705', unit=u.hour),
                     Angle('-56:44:06.3230', unit=u.deg))


        mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

#         dalt5 = dalt4+alt_grid[i]*u.deg
#         daz5 = daz4+az_grid[j]*u.deg
        dalt12 = dalt11+alt_grid[i]*u.deg
        daz12 = daz11+az_grid[j]*u.deg


        
        time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
        curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
        aa = AltAz(location=location, obstime=curr_time_atptg)
        alt_az = radec.transform_to(aa)

        cmd_elaz = AltAz(alt=alt_az.alt+dalt12, az=alt_az.az+daz12, 
                         obstime=curr_time_atptg.tai, 
                         location=location)
        cmd_radec = cmd_elaz.transform_to(FK5)


        r1.cmd_raDecTarget.set(
            targetName="Alpha Pav",
            targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
            frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
            epoch=2000,  # should be ignored: no parallax or proper motion
            equinox=2000,  # should be ignored for ICRS
            ra=cmd_radec.ra.hour,
            declination=cmd_radec.dec.deg,
            parallax=0,
            pmRA=0,
            pmDec=0,
            rv=0,
            dRA=0,
            dDec=0,
            rotPA=180.-44.,
            rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
            rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
        )

        print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


        print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

        
        await r1.cmd_raDecTarget.start()
        await wait_center()







falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


1 0
dra: -0.09324928137781541
dec: -0.4910386453603266


Center telescope and choose action (press ?<enter>, for list of actions):  s


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Skipping...
1 1
dra: -0.08834709055129508
dec: -0.5799101739205028


...

Skipping...
3 2
dra: -0.12016584904818828
dec: -0.8052213803978532


Center telescope and choose action (press ?<enter>, for list of actions):  s


RemoteEvent(ATMCS, 0, target) falling behind; read 46 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 10 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 37 elements


Skipping...
3 3
dra: -0.1150814477544202
dec: -0.8990010837017266


KeyboardInterrupt: 

In [730]:
radec = ICRS(Angle('20:25:38.85705', unit=u.hour),
             Angle('-56:44:06.3230', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt13 = dalt12+0.04*u.deg
daz13 = daz12


time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt13, az=alt_az.az+daz13, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Pav",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

ddra = radec.ra-cmd_radec.ra
dddec = radec.dec-cmd_radec.dec
print(ddra.to(u.arcsec))
print(dddec.to(u.arcsec))
print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


-6596.1arcsec
-3217.75arcsec
dra: -0.1221499175645242
dec: -0.8938205162287858


In [733]:
await r1.cmd_offsetRADec.set_start(off1=1000, off2=1000, num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [734]:
await r1.cmd_offsetRADec.set_start(off1=0., off2=217.75, num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [738]:
await r1.cmd_offsetRADec.set_start(off1=596.1, off2=0., num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


# Alpha Psa

In [ ]:
for i in range(1,len(alt_grid)):
    for j in range(len(az_grid)):
        
        print(f"{i} {j}")
        
        radec = ICRS(Angle('22:57:39.04625', unit=u.hour),
                     Angle('-29:37:20.0533', unit=u.deg))

        mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

#         dalt5 = dalt4+alt_grid[i]*u.deg
#         daz5 = daz4+az_grid[j]*u.deg
        dalt15 = dalt14+alt_grid[i]*u.deg
        daz15 = daz14+az_grid[j]*u.deg


        
        time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
        curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
        aa = AltAz(location=location, obstime=curr_time_atptg)
        alt_az = radec.transform_to(aa)

        cmd_elaz = AltAz(alt=alt_az.alt+dalt12, az=alt_az.az+daz12, 
                         obstime=curr_time_atptg.tai, 
                         location=location)
        cmd_radec = cmd_elaz.transform_to(FK5)


        r1.cmd_raDecTarget.set(
            targetName="Alpha PsA",
            targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
            frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
            epoch=2000,  # should be ignored: no parallax or proper motion
            equinox=2000,  # should be ignored for ICRS
            ra=cmd_radec.ra.hour,
            declination=cmd_radec.dec.deg,
            parallax=0,
            pmRA=0,
            pmDec=0,
            rv=0,
            dRA=0,
            dDec=0,
            rotPA=180.-44.,
            rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
            rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
        )

        print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


        print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

        
        await r1.cmd_raDecTarget.start()
        await wait_center()








In [763]:
radec = ICRS(Angle('22:57:39.04625', unit=u.hour),
             Angle('-29:37:20.0533', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt14 = dalt13+0.02*u.deg
daz14 = daz13-0.6*u.deg


time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt14, az=alt_az.az+daz14, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha PsA",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

ddra = radec.ra-cmd_radec.ra
dddec = radec.dec-cmd_radec.dec
print(ddra.to(u.arcsec))
print(dddec.to(u.arcsec))
print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 100 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling be

-5356.32arcsec
540.046arcsec
dra: -0.09919119223349071
dec: 0.15001268417304914


In [764]:
await r1.cmd_offsetRADec.set_start(off1=0., off2=-540.046, num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [770]:
await r1.cmd_offsetRADec.set_start(off1=356.32, off2=0., num=1)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


# Alpha Aqr

In [775]:
for i in range(1,len(alt_grid)):
    for j in range(len(az_grid)):
        
        print(f"{i} {j}")
        
        radec = ICRS(Angle('22:05:47.0359315', unit=u.hour),
                     Angle('-00:19:11.456766', unit=u.deg))



        mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

#         dalt5 = dalt4+alt_grid[i]*u.deg
#         daz5 = daz4+az_grid[j]*u.deg
        dalt16 = dalt15+alt_grid[i]*u.deg
        daz16 = daz15+az_grid[j]*u.deg


        
        time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
        curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
        aa = AltAz(location=location, obstime=curr_time_atptg)
        alt_az = radec.transform_to(aa)

        cmd_elaz = AltAz(alt=alt_az.alt+dalt16, az=alt_az.az+daz16, 
                         obstime=curr_time_atptg.tai, 
                         location=location)
        cmd_radec = cmd_elaz.transform_to(FK5)


        r1.cmd_raDecTarget.set(
            targetName="Alpha Aqr",
            targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
            frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
            epoch=2000,  # should be ignored: no parallax or proper motion
            equinox=2000,  # should be ignored for ICRS
            ra=cmd_radec.ra.hour,
            declination=cmd_radec.dec.deg,
            parallax=0,
            pmRA=0,
            pmDec=0,
            rv=0,
            dRA=0,
            dDec=0,
            rotPA=180.-44.,
            rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
            rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
        )

        print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


        print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

        
        await r1.cmd_raDecTarget.start()
        await wait_center()









RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, mountMotorEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTimesToLimits) falling behind; read 18 messages
RemoteTelemetry(ATMCS, 0, mountEncoders) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, target) f

1 0
dra: -0.02799194260288118
dec: 0.9121087140042214


Center telescope and choose action (press ?<enter>, for list of actions):  s


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


Skipping...
1 1
dra: -0.03361889334627577
dec: 0.8413862197281725


...

Skipping...
5 1
dra: -0.06295349410982354
dec: 1.3437015759251472


Center telescope and choose action (press ?<enter>, for list of actions):  s


RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 11 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 64 elements
falling behind; queue contains 63 elements


Skipping...
5 2
dra: -0.06834212962769115
dec: 1.2741433145344025


KeyboardInterrupt: 

In [789]:
radec = ICRS(Angle('22:05:47.0359315', unit=u.hour),
             Angle('-00:19:11.456766', unit=u.deg))

mount = await r2.tel_mountEncoders.next(flush=True, timeout=1.)

dalt17 = dalt16-0.045*u.deg
daz17 = daz16-0.04*u.deg


time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
aa = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(aa)

cmd_elaz = AltAz(alt=alt_az.alt+dalt17, az=alt_az.az+daz17, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)


r1.cmd_raDecTarget.set(
    targetName="Alpha Aqr",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-44.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)

ddra = radec.ra-cmd_radec.ra
dddec = radec.dec-cmd_radec.dec
print(ddra.to(u.arcsec))
print(dddec.to(u.arcsec))
print(f"dra: {radec.ra.hour-cmd_radec.ra.hour}")


print(f"dec: {radec.dec.deg-cmd_radec.dec.deg}")

await r1.cmd_raDecTarget.start()
# await r1.cmd_raDecTarget.start(timeout=30.)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


-3581.78arcsec
4471.65arcsec
dra: -0.0663292088962173
dec: 1.242124480162421


In [794]:
await r1.cmd_offsetRADec.set_start(off1=0., off2=-471.65, num=1)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 99 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [774]:
await r2.cmd_startTracking.start()

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling beh